### Libraries Required
- `img_as_float`, `io`, `color` from `skimage`
- `glob`
- `numpy` as `np`
- `cv2` for displaying images as video

In [2]:
import numpy as np
from skimage import img_as_float, io, color
import glob
import cv2

### Overview
This script processes a sequence of images, comparing each image to a predefined background to identify significant differences. The process involves reading a background image and a sequence of images, converting them to grayscale, computing the differences, and finally displaying the concatenated original and difference images.

### File Paths
- **Background Image Path**: `./data/Background.jpg`
- **Image Sequence Path**: `./data/video`

### Threshold
- **Difference Threshold**: `0.2`

### Detailed Steps

1. **Read the Background Image**:
   - Load the background image from the specified path.
   - Convert the image to grayscale and float format.

2. **Read Image Sequence**:
   - List and sort all JPEG files in the designated image sequence directory.
   - Convert each image to grayscale and float format for processing.

3. **Compute Differences**:
   - For each image in the sequence, compute the absolute difference from the background.
   - Check if the difference exceeds the set threshold, marking significant changes.

4. **Concatenate Images**:
   - For each image, concatenate the original and its difference image side by side.

5. **Display Results**:
   - Sequentially display each concatenated image as part of a simple video simulation.
   - Wait briefly between each image to simulate video playback.

![image](utils/1.png)

In [3]:
background_path = './data/Background.jpg'
image_sequence_path = './data/video'
threshold=0.2

background = io.imread(background_path)
background = img_as_float(color.rgb2gray(background))

# Read the images in the image sequence folder
image_list = []
for filename in glob.glob(image_sequence_path + '/*.jpg'):
    image_list.append(filename)
image_list.sort()

# Read the images in the list and convert them to float
image_sequence_list = []
for image in image_list:
    image_sequence_list.append(img_as_float(color.rgb2gray(io.imread(image))))

# Compute the difference between the background and the current image
# and check if the difference is greater than the threshold
diffrence_sequence_list = []
for image in image_sequence_list:
    diffrence = (abs(background - image) > threshold)
    diffrence_sequence_list.append(diffrence)

concat_list = []
for i in range(len(image_sequence_list)):
    concat_list.append(np.concatenate((image_sequence_list[i], diffrence_sequence_list[i]), axis=1))

# show concatenated images as video
for image in concat_list:
    cv2.imshow('image', image)
    cv2.waitKey(100)
cv2.destroyAllWindows()

### Overview
This updated script introduces a method to dynamically compute the background from a set of initial images, rather than using a single static background image. This approach may enhance the accuracy of identifying significant differences when conditions in the sequence vary slightly over time.

### Changes from the Previous Version
- **Dynamic Background Calculation**: Instead of using a pre-defined background image, the background is computed as the average of the first `init_number` images in the sequence.

### Parameters
- **Difference Threshold**: `0.2`
- **Initial Number of Images for Background**: `30`

### Detailed Steps

1. **Compute Background**:
   - Initialize a zero matrix the size of the first image.
   - Accumulate the pixel values of the first `init_number` images and average them to compute the background.

2. **Compute Differences**:
   - For each image beyond the initial set used for the background, compute the absolute difference from the computed background.
   - Check if the difference exceeds the threshold to detect significant changes.

3. **Concatenate Images**:
   - For each qualifying image, concatenate it side by side with its difference image.

4. **Display Results**:
   - Display each concatenated image sequentially as part of a video simulation.
   - Use a brief pause between frames to simulate video playback.

![image](utils/2.png)

In [4]:
threshold = 0.2
init_number = 30

# Compute the background using the first init_number images
background = np.zeros(image_sequence_list[0].shape)
for i in range(init_number):
    background += image_sequence_list[i]
background /= init_number

# Compute the difference between the background and the current image
# and check if the difference is greater than the threshold
diffrence_sequence_list = []
for i in range(init_number, len(image_sequence_list)):
    diffrence = (abs(background - image_sequence_list[i]) > threshold)
    diffrence_sequence_list.append(diffrence)

concat_list = []
for i in range(len(diffrence_sequence_list)):
    concat_list.append(np.concatenate((image_sequence_list[i + init_number], diffrence_sequence_list[i]), axis=1))

# show concatenated images as video
for image in concat_list:
    cv2.imshow('image', image)
    cv2.waitKey(100)
cv2.destroyAllWindows()

### Overview
This version of the script further improves upon the dynamic background calculation by introducing an adaptive background update mechanism. This allows the background to be incrementally adjusted based on the images that do not show significant changes, potentially increasing the robustness of the difference detection over time.

### Changes from the Previous Version
- **Adaptive Background Update**: After computing the difference for each image, the background is updated using a weighted average where the weight is controlled by the parameter `alpha`. This update is conditional, applied only to pixels that do not exceed the difference threshold.
- **Introduction of `alpha` Parameter**: Controls the rate at which the background adapts, blending the old background with the new image data where no significant changes are detected.

### Parameters
- **Initial Number of Images for Background**: `30`
- **Adaptation Rate (`alpha`)**: `0.8`
- **Difference Threshold**: `0.2`

### Detailed Steps

1. **Compute Initial Background**:
   - Calculate the initial background as the average of the first `init_number` images to establish a baseline.

2. **Process Each Image in Sequence**:
   - For each subsequent image, calculate the absolute difference from the current background.
   - Detect significant changes based on the threshold.
   - Conditionally update the background pixel-by-pixel using the formula `background[j, k] = alpha * background[j, k] + (1 - alpha) * image_sequence_list[i][j, k]` for pixels not exceeding the threshold.

3. **Concatenate Images**:
   - Concatenate each image with its difference image for visualization.

4. **Display Results**:
   - Show concatenated images in sequence, simulating a video stream with pauses to depict changes over time.

![image](utils/3.png)

In [5]:
init_number = 30
alpha = 0.8
threshold = 0.2

# Compute the background using the first init_number images
background = np.zeros(image_sequence_list[0].shape)
for i in range(init_number):
    background += image_sequence_list[i]
background /= init_number

# Compute the difference between the background and the current image and update the background
# and check if the difference is greater than the threshold
diffrence_sequence_list = []
for i in range(init_number, len(image_sequence_list)):
    # Detect changes
    diffrence = (abs(background - image_sequence_list[i]) > threshold)
    diffrence_sequence_list.append(diffrence)
    # Update the background
    for j in range(image_sequence_list[i].shape[0]):
        for k in range(image_sequence_list[i].shape[1]):
            if not diffrence[j, k]:
                background[j, k] = alpha * background[j, k] + (1 - alpha) * image_sequence_list[i][j, k]

concat_list = []
for i in range(len(diffrence_sequence_list)):
    concat_list.append(np.concatenate((image_sequence_list[i + init_number], diffrence_sequence_list[i]), axis=1))

# show concatenated images as video
for image in concat_list:
    cv2.imshow('image', image)
    cv2.waitKey(100)
cv2.destroyAllWindows()

### Overview
This script iteration introduces a second threshold for more granular control over background updates. This dual-threshold method refines how the background is updated by distinguishing between significant and minor changes, thus potentially reducing noise in the difference detection.

### Changes from the Previous Version
- **Second Threshold (`threshold_2`)**: Introduces a lower boundary to control background updates, helping to fine-tune which changes should influence the background model.
- **Selective Background Updating**: The background is now updated only when the difference between the current image and the background exceeds `threshold_2`, allowing for minor variations to be ignored.

### Parameters
- **Initial Number of Images for Background**: `30`
- **Adaptation Rate (`alpha`)**: `0.8`
- **Primary Difference Threshold**: `0.2`
- **Update Threshold (`threshold_2`)**: `0.1`

### Detailed Steps

1. **Compute Initial Background**:
   - Calculate the background as the average of the first `init_number` images to establish a baseline.

2. **Process Each Image in Sequence**:
   - For each subsequent image, compute the absolute difference from the background.
   - Update the background selectively: apply the update formula only for pixels where the difference between the current image and the background exceeds `threshold_2`.

3. **Concatenate Images**:
   - Concatenate each image with its computed difference image for visualization.

4. **Display Results**:
   - Sequentially show concatenated images, simulating a video stream with brief pauses, to illustrate how the scene evolves and how the background adapts over time.

In [6]:
init_number=30
threshold=0.2
threshold_2=0.1
alpha=0.8

# Compute the background using the first init_number images
background = np.zeros(image_sequence_list[0].shape)
for i in range(init_number):
    background += image_sequence_list[i]
background /= init_number

# Compute the difference between the background and the current image and update the background
# with the difference between image and background is greater than the threshold
diffrence_sequence_list = []
for i in range(init_number, len(image_sequence_list)):
    # Detect changes
    diffrence = (abs(background - image_sequence_list[i]) > threshold)
    diffrence_sequence_list.append(diffrence)
    # Update the background
    for j in range(image_sequence_list[i].shape[0]):
        for k in range(image_sequence_list[i].shape[1]):
            if abs(image_sequence_list[i][j, k] - background[j, k]) > threshold_2:
                background[j, k] = alpha * background[j, k] + (1 - alpha) * image_sequence_list[i][j, k]

concat_list = []
for i in range(len(diffrence_sequence_list)):
    concat_list.append(np.concatenate((image_sequence_list[i + init_number], diffrence_sequence_list[i]), axis=1))

# show concatenated images as video
for image in concat_list:
    cv2.imshow('image', image)
    cv2.waitKey(100)
cv2.destroyAllWindows()